'''
Question 2 : 
Point the camera to a chessboard pattern or any known set of reference points that lie on the
same plane. Capture a series of 10 images by changing the orientation of the camera in each
iteration. Select any 1 image, and using the image formation pipeline equation, set up the linear
equations in matrix form and solve for intrinsic and extrinsic parameters (extrinsic for that
particular orientation). You will need to make measurements of the actual 3D world points, and
mark pixel coordinates. Once you compute the Rotation matrix, you also need to compute the
angles of rotation along each axis. Choose your order of rotation based on your experimentation
setup.
'''

 Camera calibration and parameter estimation completed in Question 1
 
 Intrinsic and extrinsic parameters have been computed
 
 Rotation angles along each axis have been calculated
 
 Calibration results are stored and can be used for further processing


In [20]:
#  Importing necessary packages
import numpy as np
import cv2
from scipy.spatial.transform import Rotation
import math
from math import cos, sin, radians, sqrt

In [21]:
# Reading the rotational vectors from saved files from Question 1, Run q2 only after q1 as we have necessary files generated from q1
rot_v = []

with open('images/left/rotat_vector.txt', 'r') as f:
    for line in f :
        rot_v.append([[float(num)] for num in line.split(' ')])

np_rot_v = np.array(rot_v)

print("Rotation vector:")    
print(np_rot_v)

Rotation vector:
[[[-0.1857 ]
  [ 0.02907]
  [ 0.10722]]

 [[-0.1838 ]
  [ 0.02887]
  [ 0.11115]]

 [[-0.18689]
  [ 0.02503]
  [ 0.1073 ]]

 [[-0.18931]
  [-0.00639]
  [ 0.10485]]

 [[-0.19842]
  [-0.01795]
  [ 0.10114]]

 [[-0.20038]
  [-0.01522]
  [ 0.10452]]

 [[-0.20682]
  [-0.09657]
  [ 0.11011]]

 [[-0.21133]
  [-0.10965]
  [ 0.11204]]

 [[-0.20612]
  [-0.1303 ]
  [ 0.11095]]

 [[-0.22102]
  [-0.19021]
  [ 0.11096]]]


In [22]:
r_obj = Rotation.from_rotvec(np.array(rot_v[0]).reshape(1,3))
rot_matrix = r_obj.as_matrix()

print("ROTATIONAL MATRIX")
print(rot_matrix)

ROTATIONAL MATRIX
[[[ 0.99385344 -0.10907382  0.01892707]
  [ 0.10369655  0.97709926  0.18580651]
  [-0.03876025 -0.18270176  0.98240405]]]


In [23]:
# Creation of rotation matrix from rotation vector
xc, xs = cos(radians(np_rot_v[0][0][0])), sin(radians(np_rot_v[0][0][0]))
yc, ys = cos(radians(np_rot_v[0][1][0])), sin(radians(np_rot_v[0][1][0]))
zc, zs = cos(radians(np_rot_v[0][2][0])), sin(radians(np_rot_v[0][2][0]))

In [24]:
#Creation of  rotation matrices along x,y and z
rot_x_mtx = np.array([[1,0,0,0],[0,xc,-xs,0],[0,xs,-xc,0],[0,0,0,1]])
rot_y_mtx = np.array([[yc,0,ys,0],[0,1,0,0],[-ys,0,yc,0],[0,0,0,1]])
rot_z_mtx = np.array([[zc,-zs,0,0],[zs,zc,0,0],[0,0,1,0],[0,0,0,1]])

In [25]:
# Reading the translation vectors from saved files

tra_v = []

with open('images/left/trans_vector.txt', 'r') as f:
    for line in f :
        tra_v.append([[float(num)] for num in line.split(' ')])
        
np_tra_v = np.array(tra_v)

print("Transation vector:")    
print(np_tra_v)

Transation vector:
[[[-3.31369]
  [-0.56776]
  [20.42826]]

 [[-3.26923]
  [-0.57757]
  [20.35418]]

 [[-3.32057]
  [-0.54081]
  [20.40253]]

 [[-3.63059]
  [-0.47074]
  [20.28873]]

 [[-3.74412]
  [-0.45657]
  [20.30384]]

 [[-3.71272]
  [-0.50046]
  [20.32934]]

 [[-4.38982]
  [-0.4844 ]
  [20.01704]]

 [[-4.40781]
  [-0.44853]
  [19.97235]]

 [[-4.53733]
  [-0.45645]
  [19.73867]]

 [[-5.10888]
  [-0.37821]
  [19.46239]]]


In [26]:
# Translated matrix
tx = np_tra_v[0][0][0]
ty = np_tra_v[0][1][0]
tz = np_tra_v[0][2][0]
trans_mtx = np.array([[1,0,0,tx],[0,1,0,ty],[0,0,1,tz],[0,0,0,1]])

print("TRANSLATIONAL MATRIX")
print(trans_mtx)

TRANSLATIONAL MATRIX
[[ 1.       0.       0.      -3.31369]
 [ 0.       1.       0.      -0.56776]
 [ 0.       0.       1.      20.42826]
 [ 0.       0.       0.       1.     ]]


In [27]:
# Reading the intrinsic camera matrix from saved files

c_mtx = []
 
with open('images/left/camera_matrix.txt', 'r') as f:
    for line in f :
        c_mtx.append([float(num) for num in line.split(' ')])

print("CAMERA INTRINSIC MATRIX:")
print(c_mtx)

CAMERA INTRINSIC MATRIX:
[[665.00877, 0.0, 370.79036], [0.0, 693.47776, 165.44084], [0.0, 0.0, 1.0]]


In [28]:
# Intrinsic matrix

int_mtx = np.append(np.append(c_mtx, [[0],[0],[1]], axis=1), [np.array([0,0,0,1])], axis=0)
print('Intrinsinc mtx:')
print(int_mtx)

Intrinsinc mtx:
[[665.00877   0.      370.79036   0.     ]
 [  0.      693.47776 165.44084   0.     ]
 [  0.        0.        1.        1.     ]
 [  0.        0.        0.        1.     ]]


In [29]:
# Extrinsic matrix

ext_mtx = np.dot(rot_z_mtx, np.dot(rot_y_mtx, np.dot(rot_x_mtx, trans_mtx)))
print('Extrinsinc mtx')
print(ext_mtx)

Extrinsinc mtx
[[ 9.99998120e-01 -1.87297551e-03 -5.13428787e-04 -3.32310883e+00]
 [ 1.87134069e-03  9.99992994e-01  3.24011562e-03 -5.07767141e-01]
 [-5.07367192e-04 -3.24107033e-03 -9.99994619e-01 -2.04246287e+01]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]


In [30]:
# Camera Matrix

camera_matrix = np.dot(int_mtx, ext_mtx)
print("CAMERA MATRIX: ")
print(camera_matrix)

CAMERA MATRIX: 
[[ 6.64819393e+02 -2.44730278e+00 -3.71129799e+02 -9.78315193e+03]
 [ 1.21379390e+00  6.92936696e+02 -1.63193002e+02 -3.73119294e+03]
 [-5.07367192e-04 -3.24107033e-03 -9.99994619e-01 -1.94246287e+01]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]


In [31]:
# Calculate the inverse of the camera matrix to transform from image plane to real-world coordinates
inverse_mat = -np.linalg.inv(camera_matrix) 

# Homogeneous coordinates of two points in the image plane
project_points1 = np.array([[5],[10],[30],[1]])
project_points2 = np.array([[100],[90],[30],[1]])

# Transform points from image plane to real-world coordinates
real_dim_p1 = inverse_mat.dot(project_points1)
real_dim_p2 = inverse_mat.dot(project_points2)


In [40]:
#  this matrix below is the rotational matrix we got earlier

R = np.array([[ 0.99385344, -0.10907382,  0.01892707],
              [ 0.10369655,  0.97709926,  0.18580651],
              [-0.03876025, -0.18270176,  0.98240405]])

# Using cv2.RQDecomp3x3() to decompose the rotation matrix into Euler angles
roll_pitch_yaw = cv2.RQDecomp3x3(R)[0]

# Extracting individual Euler angles
roll = roll_pitch_yaw[0]
pitch = roll_pitch_yaw[1]
yaw = roll_pitch_yaw[2]

# Normalize Euler angles to standard ranges
def normalize_angle(angle):
    return (angle) % 180 - 100

# Convert angles from radians to degrees
roll_degrees = np.degrees(roll)
pitch_degrees = np.degrees(pitch)
yaw_degrees = np.degrees(yaw)

print("Roll (X axis rotation):", normalize_angle(roll_degrees), "degrees")
print("Pitch (Y axis rotation):", normalize_angle(pitch_degrees), "degrees")
print("Yaw (Z axis rotation):", normalize_angle(yaw_degrees), "degrees")

Roll (X axis rotation): 16.378851795859873 degrees
Pitch (Y axis rotation): 27.27427649006175 degrees
Yaw (Z axis rotation): 61.28613581953306 degrees
